In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score

from datetime import datetime
import cv2
import os
import zipfile
import shutil
from glob import glob


from tensorflow.keras.datasets import mnist
import tensorflow.keras.utils as utils

from tensorflow.keras.applications import Xception, ResNet50, InceptionV3, MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
ROOT_DIR = "/content"
DATA_ROOT_DIR = os.path.join(ROOT_DIR, "car_damage_type_0703/car_damage_type")
TRAIN_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "test")
TEST_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "train")

In [ ]:
### 코랩
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
 import shutil
 shutil.copy(os.path.join("/content/gdrive/MyDrive/FINAL PROJECT", "car_damage_type_0703.zip"),ROOT_DIR)

'/content/car_damage_type_0703.zip'

In [ ]:
if os.path.exists(DATA_ROOT_DIR):
    shutil.rmtree(DATA_ROOT_DIR)

with zipfile.ZipFile(os.path.join(ROOT_DIR, "car_damage_type_0703.zip"), "r") as target_file:
    target_file.extractall(ROOT_DIR)

In [ ]:
#### 훈련데이터
import os


train_label_name_list = os.listdir(TRAIN_DATA_ROOT_DIR)

for label_name in train_label_name_list:
    label_path = os.path.join(TRAIN_DATA_ROOT_DIR, label_name)
    file_count = sum(1 for file_name in os.listdir(label_path) if not file_name.startswith('.DS_Store'))
    print("train_label:", label_name, "=>", file_count)


#

train_label: crush => 1113
train_label: scratch => 1113
train_label: separate => 1113
train_label: breakeage => 1113


In [ ]:
import os



test_label_name_list = os.listdir(TEST_DATA_ROOT_DIR)

for label_name in test_label_name_list:
    label_path = os.path.join(TEST_DATA_ROOT_DIR, label_name)
    if os.path.isdir(label_path):  # 폴더인 경우에만 처리
        file_count = sum(1 for file_name in os.listdir(label_path) if not file_name.startswith('.DS_Store'))
        print("test_label:", label_name, "=>", file_count)


test_label: crush => 4219
test_label: scratch => 3036
test_label: separate => 2273
test_label: breakeage => 2031


inception v3

In [ ]:
IMG_WIDTH = 100
IMG_HEIGHT = 100

In [ ]:
TRAIN_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR,"train")
TEST_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR,'test')

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)
train_generator = train_data_gen.flow_from_directory(TRAIN_DATA_ROOT_DIR,
                                                     batch_size = 32,
                                                     color_mode = "rgb",
                                                     class_mode = "categorical",
                                                     target_size = (IMG_WIDTH, IMG_HEIGHT))

test_generator = train_data_gen.flow_from_directory(TEST_DATA_ROOT_DIR,
                                                     batch_size= 32,
                                                     color_mode = "rgb",
                                                     class_mode = "categorical",
                                                     target_size = (IMG_WIDTH, IMG_HEIGHT))


Found 11559 images belonging to 4 classes.
Found 4452 images belonging to 4 classes.


In [ ]:
pre_model = pre_trained_model = InceptionV3(weights = "imagenet", include_top=False,
                             input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))

pre_trained_model.summary()

87910968/87910968 [==============================] - 5s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 49, 49, 32)   864         ['input_6[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 49, 49, 32)  96          ['conv2d_4[0][0]']               
 alization)                                                                                       
                      

In [ ]:
model = Sequential()

model.add(pre_trained_model)

model.add(GlobalAveragePooling2D())  # Flatten 대신 사용

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(4, activation="softmax"))

model.compile(loss="categorical_crossentropy",
             optimizer=Adam(1e-5), metrics = ["acc"])

In [ ]:
start_time = datetime.now()

result = model.fit(train_generator, epochs= 30, validation_data=test_generator)

end_time = datetime.now()

Epoch 1/30
362/362 [==============================] - 211s 459ms/step - loss: 1.4156 - acc: 0.2861 - val_loss: 1.3888 - val_acc: 0.2709
Epoch 2/30
362/362 [==============================] - 149s 411ms/step - loss: 1.3636 - acc: 0.3240 - val_loss: 1.3725 - val_acc: 0.2835
Epoch 3/30
362/362 [==============================] - 145s 401ms/step - loss: 1.3455 - acc: 0.3462 - val_loss: 1.3677 - val_acc: 0.2803
Epoch 4/30
362/362 [==============================] - 148s 408ms/step - loss: 1.3172 - acc: 0.3620 - val_loss: 1.3470 - val_acc: 0.3118
Epoch 5/30
362/362 [==============================] - 145s 400ms/step - loss: 1.2890 - acc: 0.3950 - val_loss: 1.3293 - val_acc: 0.3428
Epoch 6/30
362/362 [==============================] - 145s 401ms/step - loss: 1.2568 - acc: 0.4195 - val_loss: 1.3028 - val_acc: 0.3780
Epoch 7/30
362/362 [==============================] - 143s 394ms/step - loss: 1.2129 - acc: 0.4462 - val_loss: 1.2710 - val_acc: 0.4135
Epoch 8/30
362/362 [============================

In [ ]:
model.evaluate(test_generator)

In [ ]:
# plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)

plt.title("LOSS")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid()

plt.plot(result.history["loss"], label="train_loss")
plt.plot(result.history["val_loss"], label="validation_loss")
plt.legend(loc="best")
# plt.show()

plt.subplot(1, 2, 2)

plt.title("acc")
plt.xlabel("epochs")
plt.ylabel("acc")
plt.grid()

plt.plot(result.history["acc"], label="train_acc")
plt.plot(result.history["val_acc"], label="validation_acc")
plt.legend(loc="best")
plt.show()


# 1차 모델

- model.add(Dense(64, activation="relu"))
- model.add(Dropout(0.5))

- model.add(Dense(128, activation="relu"))
- model.add(Dropout(0.5))

- optimizer=Adam(1e-5)

# 결과

-362/362 [==============================] - 137s 380ms/step - loss: 0.4188 - acc: 0.8614 - val_loss: 1.4945 - val_acc: 0.5717